In [13]:
import praw
from praw.models import MoreComments
import tqdm

In [22]:
reddit = praw.Reddit(
    client_id="p1dG7hgoowK4BSlUdar1WQ",
    client_secret="pEePtSnw7KMDZi6fCzkKaOth_pgKpQ",
    password="outdoortuesday",
    user_agent="Big Data by u/DISWillJayminMaya ",
    username="DISWillJayminMaya ",
)

In [23]:
print(reddit.user.me())

DISWillJayminMaya


In [25]:
tickerlist = ['GME', 'Gamestop', 'SPY', 'TWTTR', 'Twitter', 'TSLA', 'Tesla', 'AMD']

In [26]:
subreddit = reddit.subreddit('wallstreetbets')

In [27]:
hot = subreddit.hot(limit=200)
sum = [0] * len(tickerlist) # our output array
counttotal = 0 # total number of comment read
submissions_counter = 0

In [ ]:
for submissions in hot:
    if not submissions.stickied:
        submissions_counter+=1
        if submissions_counter > 5:
            comments = submissions.comments
            for comment in comments:
                if isinstance(comment, MoreComments):
                    continue
                counttotal+=1
                for i, ticker in enumerate(tickerlist):
                    if ticker in comment.body:
                        sum[i]=sum[i]+1

In [ ]:
import pandas as pd
output=pd.DataFrame(data={'Tick': tickerlist, 'Counts': sum})
print('Total comments read: ',counttotal)
print(output[output['Counts']>0])